In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

/home/ta/anaconda3/envs/dyno/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Sequence


In [2]:
train = pd.read_csv('train.csv')

In [3]:
# Re-label
train.replace(['Charged Off', 'Default', 'Fully Paid'], [1, 1, 0], inplace=True)
train.corr()

,loan_status,loan_amnt,int_rate,annual_inc
loan_status,1.000000,0.060363,0.240359,-0.063104
loan_amnt,0.060363,1.000000,0.210658,0.350495
int_rate,0.240359,0.210658,1.000000,-0.028325
annual_inc,-0.063104,0.350495,-0.028325,1.000000


In [4]:
# Convert categorical features to numerical
train.replace(['A','B','C','D','E','F','G'], [1,2,3,4,5,6,7], inplace=True)
train.replace(['RENT', 'MORTGAGE', 'OWN', 'NONE', 'OTHER', 'ANY'], [1,2,3,4,5,6], inplace=True)
train.replace([' 36 months', ' 60 months'], [1,2], inplace=True)

In [5]:
###Fill Missing values (emp_length) with mean value
train.isnull().sum()
train['emp_length'].value_counts()
train.replace(['< 1 year', '1 year', '10+ years'], ['0 years', '1 years', '10 years'], inplace=True)
train.replace(['0 years', '1 years', '2 years', '3 years', '4 years', '5 years', '6 years', '7 years', '8 years', '9 years', '10 years'],
              [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], inplace=True)

emp_length_mean = train['emp_length'].mean()
train.fillna(emp_length_mean, inplace=True)
train.corr()

,loan_status,loan_amnt,int_rate,grade,emp_length,home_ownership,annual_inc,term
loan_status,1.000000,0.060363,0.240359,0.229346,-0.003686,-0.037924,-0.063104,0.152834
loan_amnt,0.060363,1.000000,0.210658,0.208031,0.112387,0.126505,0.350495,0.402963
int_rate,0.240359,0.210658,1.000000,0.949896,0.028016,-0.045657,-0.028325,0.445961
grade,0.229346,0.208031,0.949896,1.000000,0.018977,-0.041437,-0.019112,0.467269
emp_length,-0.003686,0.112387,0.028016,0.018977,1.000000,0.170038,0.075455,0.068862
home_ownership,-0.037924,0.126505,-0.045657,-0.041437,0.170038,1.000000,0.105532,0.071636
annual_inc,-0.063104,0.350495,-0.028325,-0.019112,0.075455,0.105532,1.000000,0.069682
term,0.152834,0.402963,0.445961,0.467269,0.068862,0.071636,0.069682,1.000000


In [6]:
label = train.loan_status
X = train.drop(columns='loan_status')
X

,loan_amnt,int_rate,grade,emp_length,home_ownership,annual_inc,term
0,6000.0,9.76,2,4.0,1,33000.0,1
1,19700.0,6.62,1,5.0,1,45000.0,1
2,14000.0,6.03,1,0.0,2,75000.0,1
3,21000.0,13.11,2,2.0,1,51000.0,1
4,4500.0,16.29,4,2.0,2,96000.0,1
...,...,...,...,...,...,...,...
177928,4750.0,10.16,2,10.0,2,63000.0,1
177929,10000.0,9.17,2,10.0,1,70600.0,1
177930,9000.0,14.33,3,10.0,1,63000.0,1
177931,19800.0,22.99,6,6.0,2,100000.0,2


In [7]:
clf = DecisionTreeClassifier()

In [8]:
clf.fit(X,label)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [9]:
test = pd.read_csv('test.csv', index_col=0)
test.head()

,loan_amnt,int_rate,grade,emp_length,home_ownership,annual_inc,term
loan_status,,,,,,,
Fully Paid,9525.0,6.62,A,2 years,RENT,60000.0,36 months
Charged Off,17325.0,11.14,B,1 year,MORTGAGE,54000.0,36 months
Fully Paid,7000.0,8.49,A,5 years,RENT,118000.0,36 months
Fully Paid,10000.0,16.24,C,10+ years,MORTGAGE,150000.0,60 months
Fully Paid,9600.0,12.12,B,8 years,RENT,120000.0,36 months


In [10]:
# Pre-Processing for test data
test.replace(['< 1 year', '1 year', '10+ years'], ['0 years', '1 years', '10 years'], inplace=True)
test.replace(['0 years', '1 years', '2 years', '3 years', '4 years', '5 years', '6 years', '7 years', '8 years', '9 years', '10 years'],
              [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], inplace=True)
emp_length_mean = test['emp_length'].mean()
test.fillna(emp_length_mean, inplace=True)

In [11]:
test.replace(['A','B','C','D','E','F','G'], [1,2,3,4,5,6,7], inplace=True)
test.replace(['RENT', 'MORTGAGE', 'OWN', 'NONE', 'OTHER', 'ANY'], [1,2,3,4,5,6], inplace=True)
test.replace([' 36 months', ' 60 months'], [1,2], inplace=True)

In [12]:
#Run model

In [13]:
y_score = clf.predict(test)

In [14]:
y_test = pd.DataFrame(test.index)

In [15]:
y_test.replace(['Fully Paid', 'Charged Off', 'Default'], [0,1,1], inplace=True)

In [16]:
y_test_arr = y_test.loan_status

In [17]:
roc_auc_score = metrics.roc_auc_score(y_test_arr, y_score)

In [18]:
gini = 2*roc_auc_score - 1

In [19]:
gini

0.08409553270895431

In [20]:
from sklearn.linear_model import LinearRegression, LogisticRegression
clf1 = LinearRegression()
clf1.fit(X, label)
y_score1 = clf1.predict(test)
r1 = metrics.roc_auc_score(y_test_arr, y_score1)
g1 = 2*r1-1
g1

/home/ta/anaconda3/envs/dyno/lib/python3.7/site-packages/sklearn/model_selection/_split.py:18: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/home/ta/anaconda3/envs/dyno/lib/python3.7/site-packages/sklearn/model_selection/_search.py:16: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping, namedtuple, defaultdict, Sequence


0.3772868041602959

In [21]:
y_score1

array([0.05491267, 0.13184013, 0.06038104, ..., 0.22215343, 0.25364019,
       0.13580487])

In [24]:
clf2 = LogisticRegression()
clf2.fit(X, label)
y_score2 = clf2.predict_proba(test)[:,1]
r2 = metrics.roc_auc_score(y_test_arr, y_score2)
g2 = 2*r2-1
g2

0.18338940986771757